In [1]:
import os
import sys

# Step 2.1: Set the path to your Django app directory
sys.path.append(os.path.abspath(os.path.join('..', '..', 'backend', 'kit_hunter')))  # Adjust path as necessary

# Step 2.2: Set up Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'kit_hunter.settings')  # Adjust according to your settings module

import django
django.setup()  # Initialize Django

In [3]:
import threading

from app.models import League, Team
from django.db.models import Count  # Import Count

def get_leagues():
    leagues = League.objects.all()
    return leagues

def main():

    leagues = get_leagues()

    for league in leagues:
        teams = Team.objects.filter(league_id__exact=league.id).values('country').annotate(count=Count('id')).order_by('-count')
        print(len(teams))
        if teams:
            league.country = teams[0]["country"]
            league.save()


# Running in a separate thread
thread = threading.Thread(target=main)
thread.start()
thread.join()  # Wait for the thread to finish

2
1
2
1
1
0
1
1
1
2
1
1
1
0
0
1
2
1
1
1
1
1
1
1
0
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
2
1
1
2
1
1
1
0
1
2
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
1
0
0
1
1
0
0
0
0
1
1
1
2
1
1
2
0
0
1
1
1
1
0
0
0
1
0
0
1
1
0
0
1
1
1
1
1
0
4
0
1
0
1
3
1
1
0
1
1
0
1
0
0
0
1
0
1
1
0
1
0
1
1
1


In [4]:
import threading

from app.models import League, Team

def main():

    teams = Team.objects.all()

    for team in teams:
        team.id = team.id.replace("http://www.wikidata.org/entity/", "")
        team.save()

    teams = Team.objects.all().filter(id__contains = "http://www.wikidata.org/entity/")
    
    teams.delete()

# Running in a separate thread
thread = threading.Thread(target=main)
thread.start()
thread.join()  # Wait for the thread to finish

In [3]:
import sqlite3

def update_keys(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    try:
        # Disable foreign key constraints
        cursor.execute("PRAGMA foreign_keys = OFF;")
        
        # Retrieve all current primary keys and generate new keys
        cursor.execute("SELECT id FROM league;")
        key_mapping = {row[0]: row[0].replace("http://www.wikidata.org/entity/", "") for row in cursor.fetchall()}

        # Update primary keys in the main table
        for old_key, new_key in key_mapping.items():
            cursor.execute("UPDATE league SET id = ? WHERE id = ?", (new_key, old_key))
        
        # Update foreign keys in related tables
        for old_key, new_key in key_mapping.items():
            cursor.execute("UPDATE team SET league_id = ? WHERE league_id = ?", (new_key, old_key))

        # Commit changes
        conn.commit()

    except sqlite3.IntegrityError as e:
        print(f"An error occurred: {e}")
        conn.rollback()  # Roll back changes if there's an error

    finally:
        # Re-enable foreign key constraints
        cursor.execute("PRAGMA foreign_keys = ON;")
        conn.close()

update_keys("./kit_hunter.db")



In [2]:
import sqlite3

# Connect to your SQLite database file
conn = sqlite3.connect('../../backend/kit_hunter.db')  # Replace with your database file
cursor = conn.cursor()

# Delete the table
try:
    cursor.execute("DROP TABLE IF EXISTS app_userkitlog")
    conn.commit()
    print("Table 'app_userkitlog' deleted successfully.")
except sqlite3.Error as e:
    print(f"An error occurred: {e}")

# Close the connection
conn.close()


Table 'app_userkitlog' deleted successfully.
